In [87]:
# Install missing packages to kernel.

! pip install pandas
! pip install striprtf
! pip install scikit-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [99]:
# Import packages.

import pandas as pd 
pd.options.display.max_colwidth = None
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# CSDS-435: Project #2
# Adam Brennan and Charlie Song

## Helper Functions...

In [100]:
def get_cnn_health_tweets_dataframe():
    # read in the cnnhealth.txt file using utf-8 encoding
    with open('cnnhealth.txt', 'r', encoding='utf-8') as cnn_health_tweets_dataset:
        # initialize empty list to store tweets
        tweets = []

        # for each row in the dataset, skip the first two columns and append the tweet to the tweets list
        for row in cnn_health_tweets_dataset:
            columns = row.strip().split('|')
            tweet = '|'.join(columns[2:])
            tweets.append([tweet])

    # convert list of tweets to pandas DataFrame
    df = pd.DataFrame(tweets, columns=['tweet'])
    
    # return the resulting pandas DataFrame
    return df

In [101]:
def get_bag_of_words_matrix(tweets_list):
    # initialize CountVectorizer opbject 
    vectorizer = CountVectorizer()

    # build bag of words matrix from tweets list
    bag_of_words_matrix = vectorizer.fit_transform(tweets_list)
    
    return bag_of_words_matrix

In [107]:
def get_bag_of_words_dataframe(tweets_list):
    # initialize CountVectorizer opbject 
    vectorizer = CountVectorizer()

    # build bag of words matrix from tweets list
    bag_of_words_matrix = vectorizer.fit_transform(tweets_list)

    # convert bag of words matrix to a pandas DataFrame
    bag_of_words_df = pd.DataFrame(bag_of_words_matrix.toarray(), columns = vectorizer.get_feature_names_out())
    
    return bag_of_words_df

In [108]:
def get_tweet(tweets_df, tweet_index):
    # return the tweet at the specified index in string form
    return str(tweets_df.iloc[tweet_index])

In [109]:
def get_num_total_tokens(tweets_list):
    num_total_tokens = 0

    for tweet in tweets_list:
        num_total_tokens += len(tweet)

    return num_total_tokens

In [110]:
def get_tweets_summary_dataframe(tweets_list):
    num_tweets = len(tweets_list)
    num_total_tokens = get_num_total_tokens(tweets_list)
    num_unique_tokens = get_bag_of_words_dataframe(tweets_list).shape[1]
    num_avg_tokens_per_tweet = num_total_tokens / num_tweets

    data = [[num_tweets, num_total_tokens, num_unique_tokens, num_avg_tokens_per_tweet, 'CNN Health Tweets']]

    columns = ['# Tweets', '# Total Tokens', '# Unique Tokens', 'Avg. Tokens Per Tweet', 'Description']

    tweets_summary_dataframe = pd.DataFrame(data, columns=columns)
    
    return tweets_summary_dataframe

In [111]:
def get_cosine_similarity(bag_of_words_matrix):
    bag_of_words_matrix_cs = cosine_similarity(bag_of_words_matrix)

    return bag_of_words_matrix_cs

### Reading in the "cnnhealth.txt" file...

In [112]:
# get pandas DataFrame
tweets_df = get_cnn_health_tweets_dataframe()

# display DataFrame
tweets_df

,tweet
0,"An abundance of online info can turn us into e-hypochondriacs. Or, worse, lead us to neglect getting the care we need http://cnn.it/1L1t1Fv"
1,A plant-based diet that incorporates fish may be the key to preventing colorectal cancers: http://cnn.it/1xdpsjT http://pbs.twimg.com/media/CAARHEGWEAAJGz6.jpg
2,It doesn't take much to damage your hearing at a sports bar or nightclub. That's why a billion people are at risk. http://cnn.it/1BOphBk
3,RT @CNN: Forever young? Discover this island’s secrets to longevity on #TheWonderList w/ @BillWeirCNN http://cnn.it/1GNdmqc https://t.co/…
4,RT @CNN: Is post-traumatic stress disorder in your genes? A simple blood test may one day help tell you http://cnn.it/1xls8w5 http://t.co/…
...,...
4056,RT @EverydayHealth: Want killer abs? @JillianMichaels shows you how get them: http://at.cnn.com/ubAkAN
4057,Medicare at stake -- @sanjayguptaCNN talks about politicians' plans http://at.cnn.com/7nLSOm7
4058,Ann Romney talks about her experience with MS http://at.cnn.com/khb6keC
4059,Make sure your first marathon isn't your last! http://at.cnn.com/liehiPl


### Get tweet as list...

In [113]:
# get list of tweets
tweets_list = tweets_df['tweet'].tolist()

### Get the "bag of words" pandas DataFrame using the previously generated tweets pandas DataFrame...

In [114]:
# generate "bag of words" pandas DataFrame
bag_of_words_dataframe = get_bag_of_words_dataframe(tweets_list)

# display "bag of words" pandas DataFrame
bag_of_words_dataframe

,000,008relo,00fvvdw,00igyua,01,01tdt1o,02,03,04,05,...,zwwtu7,zxodiqo,zy53fec,zy7u11,zyam2xi,zyl,zyt5bpe,zzetat,zzgrrgf,zzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4057,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4058,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4059,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Get the tweets summary DataFrame...

In [115]:
# generate "bag of words" pandas DataFrame
tweets_summary_dataframe = get_tweets_summary_dataframe(tweets_list)

tweets_summary_dataframe

,# Tweets,# Total Tokens,# Unique Tokens,Avg. Tokens Per Tweet,Description
0,4061,441036,11264,108.602807,CNN Health Tweets


### Get cosine similarity of the bag of words...

In [116]:
bag_of_words_matrix = get_bag_of_words_matrix(tweets_list)

bag_of_words_matrix_cs = get_cosine_similarity(bag_of_words_matrix)

bag_of_words_matrix_cs

array([[1.        , 0.22222222, 0.21821789, ..., 0.1067521 , 0.0993808 ,
        0.12830006],
       [0.22222222, 1.        , 0.25458754, ..., 0.21350421, 0.1987616 ,
        0.25660012],
       [0.21821789, 0.25458754, 1.        , ..., 0.20965697, 0.29277002,
        0.25197632],
       ...,
       [0.1067521 , 0.21350421, 0.20965697, ..., 1.        , 0.28644595,
        0.36980013],
       [0.0993808 , 0.1987616 , 0.29277002, ..., 0.28644595, 1.        ,
        0.34426519],
       [0.12830006, 0.25660012, 0.25197632, ..., 0.36980013, 0.34426519,
        1.        ]])